# Dask jobqueue example

## What is Dask jobqueue? (<https://jobqueue.dask.org/>)

* deploys Dask workers on typical HPC job queueing systems

## Monte-Carlo estimate with multiple Dask batch job workers

We define a Dask jobqueue cluster with Dask workers that each have 8 CPUs and 48 GB of memory.

In [1]:
import dask, dask.distributed
import dask_jobqueue

In [2]:
cluster = dask_jobqueue.SLURMCluster(

    # Dask worker size
    cores=8, memory='48GB',
    processes=1, # Dask workers per job
    
    # SLURM job script things
    queue='cluster', walltime='00:15:00',
    
    # Dask worker network and temporary storage
    interface='ib0', local_directory='$TMPDIR'
)

client = dask.distributed.Client(cluster)
cluster.scale(jobs=1)

In [5]:
client

Client Scheduler: tcp://172.18.4.100:42503 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 1 Cores: 8 Memory: 48.00 GB


### What is a jobqueue cluster?
The above is all we need to specify to run the computation on compute node Dask workers. 
Let's have a look at what's happening in the background.

In [6]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             54643   cluster jupyterl smomw122  R      25:21      1 neshcl100 
             54699   cluster dask-wor smomw122  R       0:37      1 neshcl229 


In [7]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p cluster
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=45G
#SBATCH -t 00:15:00

/gxfs_home/geomar/smomw122/miniconda3/envs/dask_jobqueue_workshop/bin/python -m distributed.cli.dask_worker tcp://172.18.4.100:42503 --nthreads 8 --memory-limit 48.00GB --name dummy-name --nanny --death-timeout 60 --local-directory $TMPDIR --interface ib0 --protocol tcp://



In [8]:
client

Client Scheduler: tcp://172.18.4.100:42503 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 1 Cores: 8 Memory: 48.00 GB


### Let's scale up the cluster

In [9]:
cluster.scale(jobs=8)

In [17]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             54643   cluster jupyterl smomw122  R      26:44      1 neshcl100 
             54705   cluster dask-wor smomw122  R       0:28      1 neshcl251 
             54706   cluster dask-wor smomw122  R       0:28      1 neshcl251 
             54707   cluster dask-wor smomw122  R       0:28      1 neshcl251 
             54708   cluster dask-wor smomw122  R       0:28      1 neshcl251 
             54702   cluster dask-wor smomw122  R       0:59      1 neshcl229 
             54703   cluster dask-wor smomw122  R       0:59      1 neshcl230 
             54704   cluster dask-wor smomw122  R       0:59      1 neshcl230 
             54699   cluster dask-wor smomw122  R       2:00      1 neshcl229 


In [18]:
client

Client Scheduler: tcp://172.18.4.100:42503 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 8 Cores: 64 Memory: 384.00 GB


### From here everything is the same as with LocalCluster

In [19]:
import numpy, dask.array

def calculate_pi(size_in_bytes, number_of_chunks):
    
    """Calculate pi using a Monte Carlo method."""
    
    array_shape = (int(size_in_bytes / 8 / 2), 2)
    chunk_size = (int(array_shape[0] / number_of_chunks), 2)
    
    # 2D random positions array using dask.array
    xy = dask.array.random.uniform(
        low=0.0, high=1.0, size=array_shape,
        # specify chunk size, i.e. task number
        chunks=chunk_size )
  
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1 # boolean

    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size
    
    # start Dask calculation
    pi = pi.compute()

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    # display(xy)
    
    return pi

### Let's calculate again...

In [20]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.14151968
   pi error: 7.297358979307944e-05

CPU times: user 746 ms, sys: 32.6 ms, total: 778 ms
Wall time: 2.82 s


In [21]:
%time pi = calculate_pi(size_in_bytes=100_000_000_000, number_of_chunks=250) # 100 GB


from 100.0 GB randomly chosen positions
   pi estimate: 3.14155992512
   pi error: 3.2728469792964177e-05

CPU times: user 2 s, sys: 74.5 ms, total: 2.08 s
Wall time: 5.44 s


In [22]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=2_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141602444672
   pi error: 9.791082206778157e-06

CPU times: user 23.3 s, sys: 607 ms, total: 23.9 s
Wall time: 46.8 s


### We can easily scale down the cluster

In [23]:
cluster.scale(jobs=2)

In [25]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             54643   cluster jupyterl smomw122  R      29:00      1 neshcl100 
             54708   cluster dask-wor smomw122  R       2:44      1 neshcl251 
             54699   cluster dask-wor smomw122  R       4:16      1 neshcl229 


### And we can scale up the cluster whenever needed

In [33]:
cluster.scale(jobs=16)

In [36]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             54724   cluster dask-wor smomw122 PD       0:00      1 (Priority) 
             54643   cluster jupyterl smomw122  R      29:52      1 neshcl100 
             54719   cluster dask-wor smomw122  R       0:05      1 neshcl253 
             54720   cluster dask-wor smomw122  R       0:05      1 neshcl254 
             54721   cluster dask-wor smomw122  R       0:05      1 neshcl254 
             54722   cluster dask-wor smomw122  R       0:05      1 neshcl254 
             54723   cluster dask-wor smomw122  R       0:05      1 neshcl254 
             54711   cluster dask-wor smomw122  R       0:35      1 neshcl229 
             54712   cluster dask-wor smomw122  R       0:35      1 neshcl230 
             54713   cluster dask-wor smomw122  R       0:35      1 neshcl230 
             54714   cluster dask-wor smomw122  R       0:35      1 neshcl251 
             54715   cluster dask-wor smomw1

In [37]:
client

Client Scheduler: tcp://172.18.4.100:42503 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 11 Cores: 88 Memory: 528.00 GB


### Let's calculate again...

In [38]:
%time pi = calculate_pi(size_in_bytes=100_000_000_000, number_of_chunks=2_000) # 1 TB


from 100.0 GB randomly chosen positions
   pi estimate: 3.14158721024
   pi error: 5.443349793132768e-06

CPU times: user 8.15 s, sys: 214 ms, total: 8.36 s
Wall time: 8.5 s


In [39]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10_000) # 10 TB

### Note, we could also adaptively scale the jobqueue cluster!

Dask jobqueue is able to scale total worker number based on problem size. You can also specify a target duration.

In [40]:
cluster.adapt(
    minimum=2, maximum=16,
)

In [42]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.141622016
   pi error: 2.93624102067902e-05

CPU times: user 1.07 s, sys: 56 ms, total: 1.12 s
Wall time: 2.03 s


In [43]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=1_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141587770816
   pi error: 4.882773793291051e-06

CPU times: user 21.2 s, sys: 506 ms, total: 21.7 s
Wall time: 31.9 s


In [44]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10_000) # 10 TB